December 2020

Topographically corrects Dove imagery using USGS 3DEP DEM tiles.

Takes a list of Dove images, corresponding xml metadata files, USGS DEM tiles,
and the location the corrected images should be saved at.

In [ ]:
import os
import math
import geopandas as gpd
from geopandas import GeoDataFrame, GeoSeries
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal
import pandas as pd
import rasterio, rasterio.merge
import richdem as rd
from shapely.geometry import Polygon
import xml.etree.ElementTree as ET 

class Topographic_correction():
    '''
    Topographically corrects Dove imagery using USGS 3DEP DEM
    '''
    
    def __init__(self,
                #working_directory,
                dem_fps,
                image_fps,
                meta_fps,
                topo_corrected_image_fps):
        
        # Mosaic DEMs if needed
        if len(dem_fps) == 1:
            dem_fp = dem_fps[0]
        else:
            dem_fp = self.mosaic_dem(dem_fps)
            
        # Determine if DEM needs to be untiled
        dem = rasterio.open(dem_fp)
        if dem.profile['tiled'] == True:
            dem_fp = self.untile_dem(dem_fp)
        
        #dem = rasterio.open(dem_fp)
        
        # Do topographic correction
        for num, image_fp in enumerate(image_fps):
            
            # Get zenith and azimuth from Planet metadata
            zenith, az = self.get_Dove_zenith_azimuth(meta_fps[num])
            
            # Crop, reproject, and resample DEM to match image
            reproj_fp = self.crop_resample_reproject_dem(image_fp, dem_fp)
            
            # Get slope and aspect, write to files
            slope, aspect_rad = self.get_slope_aspect(reproj_fp)
            
            # Generate the cosine i
            cos_i, c1, topo_df = self.get_cosine_i(zenith, az, 
                                                   aspect_rad, 
                                                   slope,
                                                   image_fp)
            
            # Correct image and write to tif
            self.apply_topo_correction(cos_i, 
                                       c1,
                                       topo_df,
                                       image_fp,
                                       topo_corrected_image_fps[num])
        
    
    def mosaic_dem(self, dem_fps):
        '''
        Mosaics multiple DEM tiles together
        '''
        tiles = []
        
        for fp in dem_fps:
            tile = rasterio.open(fp)
            tiles.append(tile)

        full_dem, out_trans = rasterio.merge.merge(tiles)

        # Write merged DEM to file
        merged_dem_fp = 'Merged_DEM.tif'

        with rasterio.Env():
            profile = tile.profile
            profile['width'] = full_dem.shape[2]
            profile['height'] = full_dem.shape[1]
            profile['tiled'] = False
            profile['transform'] = out_trans
            
            with rasterio.open(merged_dem_fp, 'w', **profile) as dst:
                dst.write(full_dem)

        return merged_dem_fp
    
    def untile_dem(self, dem_fp):
        '''
        Removed tiling from DEM to allow for resamling to match imagery
        '''
        dem = rasterio.open(dem_fp)
        dem_array = dem.read(1)    
        untiled_fp = 'Untiled_org_dem.tif'

        with rasterio.Env():
            dem_profile = dem.profile
            dem_profile['tiled'] = False
            print(profile)
            with rasterio.open(untiled_fp, 'w', **dem_profile) as dst:
                dst.write(dem_array, 1)

        return untiled_fp  

    def get_Dove_zenith_azimuth(self, metadata_fp):
        '''
        Pull zenith and azimuth from Dove metadata xml
        '''
        # Get zenith and azimuth from Planet metadata 
        tree = ET.parse(metadata_fp)
        root = tree.getroot()

        for item in root.findall('{http://www.opengis.net/gml}using'):
            for child in item:
                for i in child.findall('{http://earth.esa.int/eop}acquisitionParameters'):
                    for j in i.findall('{http://schemas.planet.com/ps/v1/planet_product_metadata_geocorrected_level}Acquisition'):
                        for incidence_angle in j.findall('{http://earth.esa.int/eop}incidenceAngle'):
                            ia = incidence_angle.text
                            print('Incidence angle is: ', ia)
                        for illumination_azimuth in j.findall('{http://earth.esa.int/opt}illuminationAzimuthAngle'):
                            az = illumination_azimuth.text
                            print('Illumination Azimuth Angle is: ', az)
                        for illumination_elevation in j.findall('{http://earth.esa.int/opt}illuminationElevationAngle'):
                            ie = illumination_elevation.text
                            print('Illumination Elevation Angle is: ', ie)

        #  zenith angle = 90 - illumnation elevation(in degrees)
        zenith = 90 - float(ie)

        # Convert zenith and illumination azimuth to radians
        zenith = zenith * (math.pi/180)

        az = float(az) * (math.pi/180)
        
        return zenith, az
    
    def crop_resample_reproject_dem(self,image_fp, dem_fp):
        '''
        Use gdal warp to conform DEM to image.
        '''    
        # Find bounds of image to crop DEM to  
        raster = rasterio.open(image_fp)
        limits = [raster.bounds[0], raster.bounds[1], raster.bounds[2], raster.bounds[3]]

        # Reproject DEM to match image
        img_crs = raster.crs
        dem = rasterio.open(dem_fp)
        dem_crs = dem.profile['crs']

        # Generate gdal warp options 
        warp_options = gdal.WarpOptions(format='GTiff', 
                                        outputBounds=limits,
                                        xRes=3, 
                                        yRes=3,
                                        srcSRS=dem_crs,
                                        dstSRS=img_crs,
                                        resampleAlg='cubicspline',
                                        dstNodata=dem.profile['nodata'],
                                        multithread=True)

        reproj_fp = 'Reprojected_dem.tif'
        rerun = gdal.Warp(reproj_fp, dem_fp, options=warp_options)
        rerun = None
        
        return reproj_fp
    
    def get_slope_aspect(self, reproj_fp):
        '''
        Use richdem to get slope and aspect; write to tif
        '''
        reproj_dem = rasterio.open(reproj_fp)
        meta = reproj_dem.meta

        reproj_dem = rd.LoadGDAL(reproj_fp)

        slope = rd.TerrainAttribute(reproj_dem, attrib='slope_radians')

        with rasterio.open('Slope_rad.tif', "w", **meta) as dest:
            dest.write(slope, 1)

        aspect = rd.TerrainAttribute(reproj_dem, attrib='aspect')
        # Convert aspect to radians
        aspect_rad = aspect * (math.pi/180)

        with rasterio.open('Aspect_rad.tif', "w", **meta) as dest:
            dest.write(aspect_rad, 1)
            
        return slope, aspect_rad
    
    def calc_cosine_i(self, solar_zn, solar_az, aspect ,slope):
        """
        All angles are in radians
        """
        relAz = aspect - solar_az
        cosine_i = np.cos(solar_zn)*np.cos(slope) + np.sin(solar_zn)*np.sin(slope)*  np.cos(relAz)

        return cosine_i
    
    def get_cosine_i(self, zenith, az, aspect_rad, slope, image_fp):   
        '''
        Generate the cosine i.
        The cosine of the incidence angle (i), defined as the angle between 
        the normal to the pixel surface and the solar zenith direction
        '''
        print("Calculating incidence angle...")
        cos_i = self.calc_cosine_i(zenith, az, aspect_rad, slope)

        c1 = np.cos(zenith) * np.cos(slope)

        topo_coeffs = []

        raster = rasterio.open(image_fp)

        for band in range(1, raster.count+1):
            array = raster.read(band)

            #Create mask
            mask = array != 0

            # Mask cosine i image
            cos_i_masked = cos_i[mask]

            # Reshape for regression
            cos_i_reshape = np.expand_dims(cos_i_masked,axis=1)

            X = np.concatenate([cos_i_reshape,np.ones(cos_i_reshape.shape)],axis=1)

            y = array[array != 0]

            # Eq 7. Soenen et al., IEEE TGARS 2005
            slope, intercept = np.linalg.lstsq(X, y)[0].flatten()
            # Eq 8. Soenen et al., IEEE TGARS 2005
            C = intercept/slope

            # Set a large number if slope is zero
            if not np.isfinite(C):
                C = 100000.0

            topo_coeffs.append(C)

        # Store coeffs in a pandas dataframe
        topo_df =  pd.DataFrame(topo_coeffs, index=range(1, raster.count+1), columns = ['c'])

        return cos_i, c1, topo_df

    def apply_topo_correction(self, cos_i, c1, topo_df, image_fp, corrected_image_fp):
        '''
        Applies topographic correction
        '''
        raster = rasterio.open(image_fp)
        
        for band in range(1, raster.count+1):
            array = raster.read(band)

            # Apply TOPO correction 
            # Eq 11. Soenen et al., IEEE TGARS 2005
            correctionFactor = (c1 + topo_df.c.values[band-1])/(cos_i + topo_df.c.values[band-1])

            topo_array = array * correctionFactor

            # Reassign no_data values            
            topo_array[array == 0] = 0

            # Convert back to integer 
            topo_array_int = topo_array.astype('uint16')

            # Write transformed array to new image
            with rasterio.Env():
                new_profile = raster.profile
                new_profile.update(nodata=0)

                if band == 1:
                    with rasterio.open(corrected_image_fp, 'w', **new_profile) as dst:
                        dst.write(topo_array_int, band)
                else:
                    with rasterio.open(corrected_image_fp, 'r+', **new_profile) as dst:
                        dst.write(topo_array_int, band) 


In [ ]:
os.chdir('fp to directory you want the corrected image saved to')

In [ ]:
# Location of DEM
dem_fps = ['.../USGS_13_n47w094.tif',
          '.../USGS_13_n46w094.tif']

# Location of Dove images
image_fps = ['.../files/20200811_142619_0f2e_3B_AnalyticMS_SR_clip_masked.tif']

# Location of Dove metadata xml files 
meta_fps = ['.../files/20200811_142619_0f2e_3B_AnalyticMS_metadata_clip.xml']
           

# Desired names of topo-corrected images
topo_corrected_image_fps = ['Topo_corrected_Aug18.tif',
                            'Topo_corrected_Aug11.tif'
                           ]

In [ ]:
Topographic_correction(dem_fps, image_fps, meta_fps, topo_corrected_image_fps)